In [33]:
import logging
import os.path
from netCDF4 import Dataset

import numpy as np
from poap.controller import BasicWorkerThread, ThreadController

from pySOT.experimental_design import SymmetricLatinHypercube, LatinHypercube
from pySOT.optimization_problems import Hartmann6
from pySOT.strategy import EIStrategy
from pySOT.surrogate import GPRegressor

from pySOT.optimization_problems import OptimizationProblem

In [20]:
class climate_problem(OptimizationProblem):
    def __init__(self):
        self.min = 0
        self.minimum = np.array([0.0,0.0])
        self.dim = 2
        self.lb = np.array([0.01,1.02E20])
        self.ub = np.array([1.4, 1.67e24])
        self.int_var = np.array([])
        self.cont_var = np.arange(0,2)
        self.para_names = ['vdis','beta_con']
        self.info = ("wrf solar")
        self.iteration = -1
        self.archive_path = "/S2/gscr2/tzhang/big_data/UQ/tune2/"
        
    def eval(self, x):
        paras = {}
        
        for i,key in enumerate(self.para_names):
            paras[key] = x[i]
            
        mcpi = self.run_case(paras)
        return mcpi
    
    def run_case(self, x):
        self.iteration = self.iteration + 1

        # modify namelist
        for key in x:
            replace_str = "sed -i '/^ "+key+"/c\ "+key+"="+str(x[key])+"' namelist.input"
            os.system(replace_str)

        # run model
        jid = os.popen("qsub runwrf.sh").read().split('.')[0]
        #print(jid)
        while os.popen("qstat").read().find(jid) != -1:
            time.sleep(300)

        mcpi = self.calc_metrics("/home/tzhang/wrf_solar/wsolar412_bnl/runwrf_tune/wrfout_d02_2016-06-19_06:00:00")

        #archive case
        os.system("mkdir -p "+archive_path+"/Iter"+str(ite))
        os.system("cp namelist.input "+archive_path+"/Iter"+str(ite))
        os.system("cp wrfout_* "+archive_path+"/Iter"+str(ite))

        return mcpi
    
    def calc_metrics(self, path_mod):
        var_list = ['dni','dhi'] # dni: direct normal irradiance DHI: diffuse horizontal irradiance
        varn_obs = ['obs_swddni', 'obs_swddif']
        varn_mod = ['SWDDNI','SWDDIF']

        path_base = "/ss/hsdc/home/tzhang/wrf_solar/"
        path_obs = path_base+"sgpradflux10long_area_mean.c1.20160619_1200UTC.nc"
        path_def = "/S2/gscr2/tzhang/big_data/UQ/tune/runwrf_def/wrfout_d02_2016-06-19_06:00:00"

        fid_obs = Dataset(path_obs)
        fid_def = Dataset(path_def)
        fid_mod = Dataset(path_mod)

        Chi = 0
        for i, var in enumerate(var_list):
            #print(var)

            var_obs = fid_obs.variables[varn_obs[i]][:]
            var_mod = fid_mod.variables[varn_mod[i]][36:]
            var_def = fid_def.variables[varn_mod[i]][36:]

            var_mod_avg = np.mean(var_mod, axis=(1,2))
            var_def_avg = np.mean(var_def, axis=(1,2))

            #print(var_obs.shape)
            #print(var_mod_avg.shape)
            #print(var_def_avg.shape)

            theta_mod = 0
            for j in range(var_obs.shape[0]):
                theta_mod += (var_obs[j] - var_mod_avg[j]) ** 2

            theta_def = 0
            for j in range(var_obs.shape[0]):
                theta_def += (var_obs[j] - var_def_avg[j]) ** 2

            Chi += (theta_mod / theta_def)

        Chi /= len(var_list)

        return Chi
        

In [35]:
num_threads = 10
max_evals = 200

wrf = climate_problem()
#wrf = Hartmann6()
print(wrf.dim,wrf.lb,wrf.ub)

gp = GPRegressor(dim=wrf.dim, lb=wrf.lb, ub=wrf.ub)
slhd = LatinHypercube(dim=wrf.dim, num_pts=3)

# Create a strategy and a controller
controller = ThreadController()
controller.strategy = EIStrategy(
    max_evals=max_evals, opt_prob=wrf, exp_design=slhd, surrogate=gp, asynchronous=True
)

# print("Number of threads: {}".format(num_threads))
# print("Maximum number of evaluations: {}".format(max_evals))
# print("Strategy: {}".format(controller.strategy.__class__.__name__))
# print("Experimental design: {}".format(slhd.__class__.__name__))
# print("Surrogate: {}".format(gp.__class__.__name__))

# # Launch the threads and give them access to the objective function
# for _ in range(num_threads):
#     worker = BasicWorkerThread(controller, hart6.eval)
#     controller.launch_worker(worker)

# # Run the optimization strategy
# result = controller.run()

# print("Best value found: {0}".format(result.value))
# print(
#     "Best solution found: {0}\n".format(
#         np.array_str(result.params[0], max_line_width=np.inf, precision=5, suppress_small=True)
#     )
# )
    


2 [1.00e-02 1.02e+20] [1.40e+00 1.67e+24]


ValueError: No valid design found, increase num_pts?

In [12]:
hart6 = Hartmann6()
print(hart6.dim, hart6.lb, hart6.ub)

6 [0. 0. 0. 0. 0. 0.] [1. 1. 1. 1. 1. 1.]


In [6]:
print(result.params)

(array([3.99977194e-01, 8.81107545e-01, 9.15603147e-01, 5.71582306e-01,
       1.05019138e-01, 1.50607404e-08]),)


In [7]:
dir(result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_status',
 'add_callback',
 'callbacks',
 'cancel',
 'complete',
 'ev_id',
 'extra_args',
 'is_cancelled',
 'is_completed',
 'is_done',
 'is_killed',
 'is_pending',
 'is_running',
 'kill',
 'params',
 'remove_callback',
 'running',
 'status',
 'update',
 'update_dict',
 'value',
 'worker']

In [10]:
print(result.status)

completed
